In [8]:
import json
import os
from typing import List, Dict, Any

# Configuración
STRAICO_DIR = 'Straico'
OUTPUT_DIR = 'Metricas LLM/Data_for_Metrics'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Cargar modelos desde el archivo text_model_ids.txt
MODELS_FILE = 'Straico/text_model_usefull.txt'

def load_models_from_file(filepath):
    """Carga los nombres de modelos desde un archivo de texto."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo {filepath}")
        return []

DEFAULT_MODELS = load_models_from_file(MODELS_FILE)

print(f"Carpeta de entrada: {STRAICO_DIR}/")
print(f"Carpeta de salida: {OUTPUT_DIR}/")
print(f"Modelos: {DEFAULT_MODELS}")

Carpeta de entrada: Straico/
Carpeta de salida: Metricas LLM/Data_for_Metrics/
Modelos: ['amazon/nova-micro-v1', 'microsoft/phi-4', 'amazon/nova-lite-v1', 'cohere/command-r-08-2024', 'qwen/qwen-2.5-72b-instruct', 'google/gemma-2-27b-it', 'meta-llama/llama-3.3-70b-instruct', 'microsoft/wizardlm-2-8x22b', 'meta-llama/llama-4-maverick', 'qwen/qwen2.5-vl-32b-instruct:free', 'x-ai/grok-3-mini-beta', 'perplexity/sonar', 'mistralai/mistral-medium-3', 'mistralai/mixtral-8x7b-instruct', 'google/gemini-2.5-flash', 'meta-llama/llama-3.1-405b-instruct', 'deepseek/deepseek-chat-v3.1', 'moonshotai/kimi-k2-0905', 'openai/o4-mini-high', 'openai/gpt-4.1', 'openai/o1-mini', 'anthropic/claude-sonnet-4']


## Cargar Dataset Original (Ground Truth)

In [9]:
import csv

def load_ground_truth(csv_path: str) -> Dict[str, Dict[str, str]]:
    """Carga el dataset original y elimina duplicados.
    
    Returns:
        Dict con modismo como clave y dict con {significado, ejemplo} como valor
    """
    ground_truth = {}
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            modismo = row.get('modismo', '').strip()
            if modismo and modismo not in ground_truth:
                ground_truth[modismo] = {
                    'significado': row.get('significado', '').strip(),
                    'ejemplo': row.get('ejemplo', '').strip()
                }
    
    print(f"Dataset cargado: {len(ground_truth)} modismos únicos")
    return ground_truth

# Cargar dataset
ground_truth = load_ground_truth('modismos_Dataset_Cleaned.csv')

Dataset cargado: 4616 modismos únicos


## Funciones Auxiliares

In [10]:
def load_prompt_responses(prompt_dir: str) -> Dict[str, List[Dict]]:
    """Carga las respuestas de todos los modelos para un prompt.
    
    Args:
        prompt_dir: Directorio del prompt (ej: 'Straico/Prompt 1')
        
    Returns:
        Dict con modelo como clave y lista de respuestas como valor
    """

    all_models_path = os.path.join(prompt_dir, 'all_models.json')
    
    if os.path.exists(all_models_path):
        with open(all_models_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    
    return {}


def extract_output_field(response: Dict, field: str) -> str:
    """Extrae un campo del output de la respuesta del modelo.
    
    Maneja múltiples formatos:
    1. Formato directo: {"output": {"campo": "valor"}}
    2. Formato raw_response: {"raw_response": "```json\\n{...}\\n```"}
    3. Errores: {"error": "..."}
    
    Args:
        response: Dict con la respuesta completa
        field: Campo a extraer del output
        
    Returns:
        Valor del campo como string, vacío si hay error o no se encuentra
    """
    try:
        if not isinstance(response, dict):
            return ''
        
        # Verificar si hay error
        if 'error' in response:
            return ''
        
        # Caso 1: Formato directo con output
        output = response.get('output', {})
        if isinstance(output, dict) and field in output:
            return str(output.get(field, '')).strip()
        
        # Caso 2: raw_response con JSON anidado
        raw_response = response.get('raw_response', '')
        if raw_response:
            # Limpiar markdown code blocks
            raw_response = raw_response.strip()
            if raw_response.startswith('```json'):
                raw_response = raw_response[7:]  # Remover ```json
            if raw_response.startswith('```'):
                raw_response = raw_response[3:]  # Remover ```
            if raw_response.endswith('```'):
                raw_response = raw_response[:-3]  # Remover ```
            
            raw_response = raw_response.strip()
            
            # Parsear el JSON anidado
            try:
                parsed = json.loads(raw_response)
                if isinstance(parsed, dict):
                    output = parsed.get('output', {})
                    if isinstance(output, dict) and field in output:
                        return str(output.get(field, '')).strip()
            except json.JSONDecodeError:
                return ''
        
        return ''
    except Exception as e:
        print(f"⚠ Error extrayendo campo '{field}': {e}")
        return ''


def save_json(filepath: str, data: Any):
    """Guarda datos en formato JSON."""
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✓ Guardado: {filepath} ({len(data)} registros)")

## PROMPT 1: Modismo → Definición

Procesa las respuestas del Prompt 1 y genera datos para métricas comparando las definiciones generadas con el ground truth.

In [11]:
print("=" * 60)
print("PROCESANDO PROMPT 1: Modismo → Definición")
print("=" * 60)

# Cargar respuestas
prompt_1_responses = load_prompt_responses(os.path.join(STRAICO_DIR, 'Prompt 1'))

# Procesar datos
prompt_1_data = []
errors_count = 0
skipped_models = []

for model in DEFAULT_MODELS:
    if model not in prompt_1_responses:
        print(f"⚠ Advertencia: Modelo {model} no encontrado en Prompt 1")
        skipped_models.append(model)
        continue
    
    model_errors = 0
    for entry in prompt_1_responses[model]:
        modismo = entry.get('modismo', '')
        response = entry.get('response', {})
        
        # Verificar si hay error en la respuesta
        if 'error' in response:
            model_errors += 1
            errors_count += 1
            continue
        
        # Extraer definición generada del output
        definicion_generada = extract_output_field(response, 'definicion')
        
        # Buscar en ground truth
        gt = ground_truth.get(modismo, {})
        definicion_real = gt.get('significado', '')
        
        # Solo agregar si se extrajo la definición correctamente
        if definicion_generada:
            prompt_1_data.append({
                'modismo': modismo,
                'definicion_real': definicion_real,
                'modelo': model,
                'definicion_generada': definicion_generada
            })
        else:
            model_errors += 1
            errors_count += 1
    
    if model_errors > 0:
        print(f"  {model}: {model_errors} errores omitidos")

# Guardar
output_path = os.path.join(OUTPUT_DIR, 'prompt_1_metrics_data.json')
save_json(output_path, prompt_1_data)

print(f"\n✓ Procesados {len(prompt_1_data)} registros válidos")
print(f"✗ Omitidos {errors_count} registros con errores")
if skipped_models:
    print(f"⚠ Modelos no encontrados: {', '.join(skipped_models)}")
print()

PROCESANDO PROMPT 1: Modismo → Definición
  amazon/nova-micro-v1: 14 errores omitidos
  microsoft/phi-4: 164 errores omitidos
  amazon/nova-lite-v1: 43 errores omitidos
  cohere/command-r-08-2024: 112 errores omitidos
  qwen/qwen-2.5-72b-instruct: 578 errores omitidos
  google/gemma-2-27b-it: 97 errores omitidos
  meta-llama/llama-3.3-70b-instruct: 36 errores omitidos
  microsoft/wizardlm-2-8x22b: 156 errores omitidos
  meta-llama/llama-4-maverick: 83 errores omitidos
  qwen/qwen2.5-vl-32b-instruct:free: 3915 errores omitidos
  x-ai/grok-3-mini-beta: 89 errores omitidos
  perplexity/sonar: 178 errores omitidos
  mistralai/mistral-medium-3: 117 errores omitidos
  mistralai/mixtral-8x7b-instruct: 138 errores omitidos
  google/gemini-2.5-flash: 115 errores omitidos
  meta-llama/llama-3.1-405b-instruct: 64 errores omitidos
  deepseek/deepseek-chat-v3.1: 19 errores omitidos
  moonshotai/kimi-k2-0905: 1990 errores omitidos
  openai/o4-mini-high: 21 errores omitidos
  openai/gpt-4.1: 19 error

## PROMPT 2: Modismo → Es Modismo (Sí/No)

Procesa las respuestas del Prompt 2 para evaluar si los modelos identifican correctamente los modismos.

In [12]:
print("=" * 60)
print("PROCESANDO PROMPT 2: Modismo → Es Modismo (Sí/No)")
print("=" * 60)

# Cargar respuestas
prompt_2_responses = load_prompt_responses(os.path.join(STRAICO_DIR, 'Prompt 2'))

# Procesar datos
prompt_2_data = []
errors_count = 0
skipped_models = []

for model in DEFAULT_MODELS:
    if model not in prompt_2_responses:
        print(f"⚠ Advertencia: Modelo {model} no encontrado en Prompt 2")
        skipped_models.append(model)
        continue
    
    model_errors = 0
    for entry in prompt_2_responses[model]:
        modismo = entry.get('modismo', '')
        response = entry.get('response', {})
        
        # Verificar si hay error en la respuesta
        if 'error' in response:
            model_errors += 1
            errors_count += 1
            continue
        
        # Extraer respuesta (Sí/No) del output
        es_modismo_generado = extract_output_field(response, 'es_modismo')
        
        # Solo agregar si se extrajo la respuesta correctamente
        if es_modismo_generado:
            prompt_2_data.append({
                'modismo': modismo,
                'es_modismo_real': 'Sí',  # Todos los del dataset son modismos
                'modelo': model,
                'es_modismo_generado': es_modismo_generado
            })
        else:
            model_errors += 1
            errors_count += 1
    
    if model_errors > 0:
        print(f"  {model}: {model_errors} errores omitidos")

# Guardar
output_path = os.path.join(OUTPUT_DIR, 'prompt_2_metrics_data.json')
save_json(output_path, prompt_2_data)

print(f"\n✓ Procesados {len(prompt_2_data)} registros válidos")
print(f"✗ Omitidos {errors_count} registros con errores")
if skipped_models:
    print(f"⚠ Modelos no encontrados: {', '.join(skipped_models)}")
print()

PROCESANDO PROMPT 2: Modismo → Es Modismo (Sí/No)
  amazon/nova-micro-v1: 19 errores omitidos
  microsoft/phi-4: 440 errores omitidos
  amazon/nova-lite-v1: 24 errores omitidos
  cohere/command-r-08-2024: 21 errores omitidos
  qwen/qwen-2.5-72b-instruct: 462 errores omitidos
  google/gemma-2-27b-it: 40 errores omitidos
  meta-llama/llama-3.3-70b-instruct: 14 errores omitidos
  microsoft/wizardlm-2-8x22b: 27 errores omitidos
  meta-llama/llama-4-maverick: 30 errores omitidos
  qwen/qwen2.5-vl-32b-instruct:free: 3784 errores omitidos
  x-ai/grok-3-mini-beta: 49 errores omitidos
  perplexity/sonar: 70 errores omitidos
  mistralai/mistral-medium-3: 96 errores omitidos
  mistralai/mixtral-8x7b-instruct: 2034 errores omitidos
  google/gemini-2.5-flash: 46 errores omitidos
  meta-llama/llama-3.1-405b-instruct: 481 errores omitidos
  deepseek/deepseek-chat-v3.1: 15 errores omitidos
  moonshotai/kimi-k2-0905: 195 errores omitidos
  openai/o4-mini-high: 20 errores omitidos
  openai/gpt-4.1: 14 e

## PROMPT 3: Modismo + Ejemplo → Literal + Definición

Procesa las respuestas del Prompt 3 para evaluar la capacidad de los modelos de generar interpretaciones literales.

In [13]:
print("=" * 60)
print("PROCESANDO PROMPT 3: Modismo + Ejemplo → Literal + Definición")
print("=" * 60)

# Cargar respuestas
prompt_3_responses = load_prompt_responses(os.path.join(STRAICO_DIR, 'Prompt 3'))

# Procesar datos
prompt_3_data = []
errors_count = 0
skipped_models = []

for model in DEFAULT_MODELS:
    if model not in prompt_3_responses:
        print(f"⚠ Advertencia: Modelo {model} no encontrado en Prompt 3")
        skipped_models.append(model)
        continue
    
    model_errors = 0
    for entry in prompt_3_responses[model]:
        modismo = entry.get('modismo', '')
        ejemplo = entry.get('ejemplo', '')
        response = entry.get('response', {})
        
        # Verificar si hay error en la respuesta
        if 'error' in response:
            model_errors += 1
            errors_count += 1
            continue
        
        # Extraer literal y definición del output
        literal_generado = extract_output_field(response, 'literal')
        definicion_generada = extract_output_field(response, 'definicion')
        
        # Buscar en ground truth
        gt = ground_truth.get(modismo, {})
        
        # Solo agregar si se extrajo al menos un campo correctamente
        if literal_generado or definicion_generada:
            prompt_3_data.append({
                'modismo': modismo,
                'ejemplo': ejemplo,
                'significado_real': gt.get('significado', ''),
                'modelo': model,
                'literal_generado': literal_generado,
                'definicion_generada': definicion_generada
            })
        else:
            model_errors += 1
            errors_count += 1
    
    if model_errors > 0:
        print(f"  {model}: {model_errors} errores omitidos")

# Guardar
output_path = os.path.join(OUTPUT_DIR, 'prompt_3_metrics_data.json')
save_json(output_path, prompt_3_data)

print(f"\n✓ Procesados {len(prompt_3_data)} registros válidos")
print(f"✗ Omitidos {errors_count} registros con errores")
if skipped_models:
    print(f"⚠ Modelos no encontrados: {', '.join(skipped_models)}")
print()

PROCESANDO PROMPT 3: Modismo + Ejemplo → Literal + Definición
  amazon/nova-micro-v1: 97 errores omitidos
  microsoft/phi-4: 396 errores omitidos
  amazon/nova-lite-v1: 63 errores omitidos
  cohere/command-r-08-2024: 40 errores omitidos
  qwen/qwen-2.5-72b-instruct: 735 errores omitidos
  google/gemma-2-27b-it: 169 errores omitidos
  meta-llama/llama-3.3-70b-instruct: 44 errores omitidos
  microsoft/wizardlm-2-8x22b: 29 errores omitidos
  meta-llama/llama-4-maverick: 131 errores omitidos
  qwen/qwen2.5-vl-32b-instruct:free: 4106 errores omitidos
  x-ai/grok-3-mini-beta: 139 errores omitidos
  perplexity/sonar: 107 errores omitidos
  mistralai/mistral-medium-3: 92 errores omitidos
  mistralai/mixtral-8x7b-instruct: 74 errores omitidos
  google/gemini-2.5-flash: 15 errores omitidos
  meta-llama/llama-3.1-405b-instruct: 186 errores omitidos
  deepseek/deepseek-chat-v3.1: 26 errores omitidos
  moonshotai/kimi-k2-0905: 291 errores omitidos
  openai/o4-mini-high: 59 errores omitidos
  openai

## Resumen de Archivos Generados

In [14]:
print("=" * 60)
print("RESUMEN DE ARCHIVOS GENERADOS")
print("=" * 60)

import glob

# Listar todos los archivos JSON generados
json_files = glob.glob(os.path.join(OUTPUT_DIR, '*.json'))
json_files.sort()

total_registros = 0
for filepath in json_files:
    filename = os.path.basename(filepath)
    
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    num_registros = len(data)
    total_registros += num_registros
    
    print(f"  {filename}: {num_registros} registros")

print("=" * 60)
print(f"Total: {len(json_files)} archivos, {total_registros} registros")
print(f"Ubicación: {OUTPUT_DIR}/")
print("\n✓ Procesamiento completado")

RESUMEN DE ARCHIVOS GENERADOS
  prompt_1_metrics_data.json: 93568 registros
  prompt_2_metrics_data.json: 93633 registros
  prompt_3_metrics_data.json: 94534 registros
Total: 3 archivos, 281735 registros
Ubicación: Metricas LLM/Data_for_Metrics/

✓ Procesamiento completado
